In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/univ/4/nlp/gpt

KeyboardInterrupt: 

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import pandas as pd
import time

# OpenAI API 키 설정
api_key = 'YOUR_API'
client = OpenAI(api_key=api_key)

def get_gpt3_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # GPT-3.5 모델 엔진 이름
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=4096,  # 원하는 응답 길이
        n=1,  # 하나의 응답 생성
        stop=None,
        temperature=0.7
    )
    return response.choices[0].message.content

# CSV 파일에서 프롬프트 읽기
csv_file = 'test_data_with_retrieve_normalized.csv'
df = pd.read_csv(csv_file)

# 응답 저장을 위한 리스트 초기화
responses = []
start_time = time.time()

# 각 프롬프트에 대해 GPT-3.5 응답 받기
for index, row in df.iterrows():
    prompt = '다음은 내가 처한 상황이야.'+row['query']+'다음은 나와 유사한 상황 판례야.'+row['fact']+'변호사로서 내 상황에 적용할 수 있는 법령과 그 이유를 설명해줘.'
    response = get_gpt3_response(prompt)
    responses.append(response)
    if (index + 1) % 10 == 0:
        elapsed_time = time.time() - start_time
        print(f'Processed {index + 1} prompts. Elapsed time: {elapsed_time:.2f} seconds.')

# 응답을 새로운 열로 데이터프레임에 추가
df['response'] = responses

# 결과를 새로운 CSV 파일로 저장
output_csv_file = 'baseline2_gpt_result.csv'
df.to_csv(output_csv_file, index=False)

print(f'Responses saved to {output_csv_file}')

Processed 10 prompts. Elapsed time: 102.14 seconds.
Processed 20 prompts. Elapsed time: 218.39 seconds.
Processed 30 prompts. Elapsed time: 315.04 seconds.
Processed 40 prompts. Elapsed time: 421.23 seconds.
Processed 50 prompts. Elapsed time: 515.05 seconds.
Processed 60 prompts. Elapsed time: 642.68 seconds.
Processed 70 prompts. Elapsed time: 739.62 seconds.
Processed 80 prompts. Elapsed time: 833.90 seconds.
Processed 90 prompts. Elapsed time: 943.74 seconds.
Processed 100 prompts. Elapsed time: 1057.97 seconds.
Processed 110 prompts. Elapsed time: 1156.32 seconds.
Processed 120 prompts. Elapsed time: 1252.38 seconds.
Processed 130 prompts. Elapsed time: 1375.10 seconds.
Processed 140 prompts. Elapsed time: 1482.89 seconds.
Processed 150 prompts. Elapsed time: 1589.34 seconds.
Processed 160 prompts. Elapsed time: 1700.39 seconds.
Processed 170 prompts. Elapsed time: 1816.28 seconds.
Processed 180 prompts. Elapsed time: 1906.15 seconds.
Processed 190 prompts. Elapsed time: 2020.93 s

In [ ]:
#baseline2-2 gpt
import pandas as pd
import numpy as np
from metrics import exact_match_score, f1, qa_f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

csv_file = 'baseline2_gpt_result.csv'
df = pd.read_csv(csv_file)

# query와 response를 group으로 묶음
df['group'] = df.index // 5

# 각 query마다 최고 점수의 response 선택
best_responses = []

# BLEU 및 ROUGE 계산 준비
smoothing = SmoothingFunction().method4
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# 각 메트릭 점수를 저장할 리스트
max_f1_scores = []
max_bleu_scores = []
max_bleu1_scores = []
max_bleu2_scores = []
max_rouge1_scores = []
max_rouge2_scores = []
max_rougeL_scores = []

for group_id, group in df.groupby('group'):
    ground_truth = group['laws'].iloc[0]  # 같은 query에 대한 ground_truth는 동일하므로 첫번째 값을 사용

    max_f1 = -1
    max_bleu = -1
    max_bleu1 = -1
    max_bleu2 = -1
    max_rouge1 = -1
    max_rouge2 = -1
    max_rougeL = -1

    for index, row in group.iterrows():
        response = row['response']

        # F1 점수 계산
        current_f1 = qa_f1_score(response, ground_truth)
        if current_f1 > max_f1:
            max_f1 = current_f1

        # BLEU 점수 계산
        current_bleu = sentence_bleu([ground_truth.split()], response.split(), smoothing_function=smoothing)
        if current_bleu > max_bleu:
            max_bleu = current_bleu

        current_bleu1 = sentence_bleu([ground_truth.split()], response.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing)
        if current_bleu1 > max_bleu1:
            max_bleu1 = current_bleu1

        current_bleu2 = sentence_bleu([ground_truth.split()], response.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
        if current_bleu2 > max_bleu2:
            max_bleu2 = current_bleu2

        # ROUGE 점수 계산
        scores = scorer.score(ground_truth, response)
        if scores['rouge1'].fmeasure > max_rouge1:
            max_rouge1 = scores['rouge1'].fmeasure
        if scores['rouge2'].fmeasure > max_rouge2:
            max_rouge2 = scores['rouge2'].fmeasure
        if scores['rougeL'].fmeasure > max_rougeL:
            max_rougeL = scores['rougeL'].fmeasure

    # 최고 점수를 리스트에 추가
    max_f1_scores.append(max_f1)
    max_bleu_scores.append(max_bleu)
    max_bleu1_scores.append(max_bleu1)
    max_bleu2_scores.append(max_bleu2)
    max_rouge1_scores.append(max_rouge1)
    max_rouge2_scores.append(max_rouge2)
    max_rougeL_scores.append(max_rougeL)

# 각 메트릭의 평균 점수 계산
avg_f1_score = 100 * np.mean(max_f1_scores)
avg_bleu_score = 100 * np.mean(max_bleu_scores)
avg_bleu1_score = 100 * np.mean(max_bleu1_scores)
avg_bleu2_score = 100 * np.mean(max_bleu2_scores)
avg_rouge1_score = 100 * np.mean(max_rouge1_scores)
avg_rouge2_score = 100 * np.mean(max_rouge2_scores)
avg_rougeL_score = 100 * np.mean(max_rougeL_scores)

# 결과 출력
print(f"F1 Score: {avg_f1_score}%")
print(f"BLEU Score: {avg_bleu_score}%")
print(f"BLEU-1 Score: {avg_bleu1_score}%")
print(f"BLEU-2 Score: {avg_bleu2_score}%")
print(f"ROUGE-1 Score: {avg_rouge1_score}%")
print(f"ROUGE-2 Score: {avg_rouge2_score}%")
print(f"ROUGE-L Score: {avg_rougeL_score}%")

F1 Score: 5.174067875261229%
BLEU Score: 0.48563415346577243%
BLEU-1 Score: 3.5441472490583354%
BLEU-2 Score: 1.2658111757854589%
ROUGE-1 Score: 22.961889440718974%
ROUGE-2 Score: 8.110033393499155%
ROUGE-L Score: 21.095906176468997%


In [ ]:
!pip install nltk rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=85260b81b77e4ef5ee4cc78d697b85e3d43771043c0290106b5b7af6b40ec213
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
#baseline2-2 gpt_inter
import pandas as pd
import numpy as np
from metrics import exact_match_score, f1, qa_f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# CSV 파일 읽기
alpha_csv_file = 'AlphaMist_total_result.csv'
alpha_df = pd.read_csv(alpha_csv_file)
csv_file = 'baseline2_gpt_result.csv'
df = pd.read_csv(csv_file)
alpha3_csv_file='baseline3_AlphaMist_result.csv'
alpha3_df=pd.read_csv(alpha3_csv_file)


# 유효한 인덱스 필터링
valid_indices = alpha_df[alpha_df['답변'].notna() & alpha_df['RAG  답변'].notna()&alpha3_df['response'].notna()].index
# group 컬럼 추가
df['group'] = df.index // 5

# BLEU 및 ROUGE 계산 준비
smoothing = SmoothingFunction().method4
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# 각 메트릭 점수를 저장할 리스트
max_f1_scores = []
max_bleu_scores = []
max_bleu1_scores = []
max_bleu2_scores = []
max_rouge1_scores = []
max_rouge2_scores = []
max_rougeL_scores = []

# 각 valid_indices에 대해 계산된 응답의 수를 저장할 리스트
num_responses = []

for idx in valid_indices:
    alpha_query = alpha_df.loc[idx, '자연어 쿼리']
    group = df[df['group'] == idx]

    # 쿼리가 같은지 확인
    df_query = group['query'].unique()
    if len(df_query) == 1 and alpha_query == df_query[0]:
        ground_truth = group['laws'].iloc[0]  # 같은 query에 대한 ground_truth는 동일하므로 첫번째 값을 사용
        max_f1 = -1
        max_bleu = -1
        max_bleu1 = -1
        max_bleu2 = -1
        max_rouge1 = -1
        max_rouge2 = -1
        max_rougeL = -1

        response_count = 0  # 현재 쿼리에 대해 계산된 응답의 수

        for index, row in group.iterrows():
            response = row['response']
            response_count += 1

            # F1 점수 계산
            current_f1 = qa_f1_score(response, ground_truth)
            if current_f1 > max_f1:
                max_f1 = current_f1

            # BLEU 점수 계산
            current_bleu = sentence_bleu([ground_truth.split()], response.split(), smoothing_function=smoothing)
            if current_bleu > max_bleu:
                max_bleu = current_bleu

            current_bleu1 = sentence_bleu([ground_truth.split()], response.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing)
            if current_bleu1 > max_bleu1:
                max_bleu1 = current_bleu1

            current_bleu2 = sentence_bleu([ground_truth.split()], response.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
            if current_bleu2 > max_bleu2:
                max_bleu2 = current_bleu2

            # ROUGE 점수 계산
            scores = scorer.score(ground_truth, response)
            if scores['rouge1'].fmeasure > max_rouge1:
                max_rouge1 = scores['rouge1'].fmeasure
            if scores['rouge2'].fmeasure > max_rouge2:
                max_rouge2 = scores['rouge2'].fmeasure
            if scores['rougeL'].fmeasure > max_rougeL:
                max_rougeL = scores['rougeL'].fmeasure

        # 최고 점수를 리스트에 추가
        max_f1_scores.append(max_f1)
        max_bleu_scores.append(max_bleu)
        max_bleu1_scores.append(max_bleu1)
        max_bleu2_scores.append(max_bleu2)
        max_rouge1_scores.append(max_rouge1)
        max_rouge2_scores.append(max_rouge2)
        max_rougeL_scores.append(max_rougeL)

        # 응답 수를 리스트에 추가
        num_responses.append(response_count)

# 각 메트릭의 평균 점수 계산
avg_f1_score = 100 * np.mean(max_f1_scores)
avg_bleu_score = 100 * np.mean(max_bleu_scores)
avg_bleu1_score = 100 * np.mean(max_bleu1_scores)
avg_bleu2_score = 100 * np.mean(max_bleu2_scores)
avg_rouge1_score = 100 * np.mean(max_rouge1_scores)
avg_rouge2_score = 100 * np.mean(max_rouge2_scores)
avg_rougeL_score = 100 * np.mean(max_rougeL_scores)

# 결과 출력
print(f"F1 Score: {avg_f1_score}%")
print(f"BLEU Score: {avg_bleu_score}%")
print(f"BLEU-1 Score: {avg_bleu1_score}%")
print(f"BLEU-2 Score: {avg_bleu2_score}%")
print(f"ROUGE-1 Score: {avg_rouge1_score}%")
print(f"ROUGE-2 Score: {avg_rouge2_score}%")
print(f"ROUGE-L Score: {avg_rougeL_score}%")
print(f"Number of queries processed: {len(num_responses)}")
print(f"Number of responses per query: {num_responses}")


F1 Score: 5.618963289658072%
BLEU Score: 0.4758430198439393%
BLEU-1 Score: 3.8015642327853545%
BLEU-2 Score: 1.3356593018791605%
ROUGE-1 Score: 26.37654900344042%
ROUGE-2 Score: 9.636342962024552%
ROUGE-L Score: 23.574246870838%
Number of queries processed: 62
Number of responses per query: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
